In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 

import collections
import os

## Load data


In [3]:
#file = '/home/parallels/Desktop/Parallels Shared Folders/Home/Desktop/CCSI2/RPB_sensitivity_8_25_22.xlsx'
file = '/Users/wangjialu/Desktop/CCSI2/RPB_sensitivity_8_25_22.xlsx'

data = pd.ExcelFile(file)

In [4]:
df1 = pd.read_excel(data, 'MTC')
df2 = pd.read_excel(data, 'HTC')
df3 = pd.read_excel(data, 'DH')
df4 = pd.read_excel(data, 'Iso_1')
df5 = pd.read_excel(data, 'Iso_2')

In [5]:
col_names_up = df1.columns.values[4:18].tolist()
col_names_lo = df1.columns.values[22:].tolist()
print('===Measurement names===')
print(col_names_up)
print(col_names_lo)

===Measurement names===
['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
['Ads.gas_inlet.F.1', 'Ads.gas_outlet.F.1', 'Ads.gas_outlet.T.1', 'Ads.gas_outlet.z("CO2").1', 'Des.gas_inlet.F.1', 'Des.gas_outlet.F.1', 'Des.gas_outlet.T.1', 'Des.gas_outlet.z("CO2").1', 'Ads.T_g.Value(19,10).1', 'Ads.T_g.Value(23,10).1', 'Ads.T_g.Value(28,10).1', 'Ads.z("CO2",19,10).1', 'Ads.z("CO2",23,10).1', 'Ads.z("CO2",28,10).1']


In [6]:
timeSet = list(df1['Time'][1:].tolist())
print(timeSet[0], timeSet[-1])
print(len(timeSet))

0 220
2201


In [7]:
# 10 scenarios in total. We are going to map them from 0 to 9 with
# MTC_up, MTC_lo, HTC_up, HTC_lo, DH_up, DH_lo, LSO1_up, LSO1_do, LSO1_up, LSO1_do

# output record: key: 1-10 scenario numbers; Value: a list of 14 lists, each list contains the data of one measurement
output_record = {}
# output_doe: key: 1-10 scenario numbers; Value: a large list containing all data of measurements
output_doe = {}

for s in [0,2,4,6,8]:
    
    df = [df1, df2, df3, df4, df5][int(s/2)]
    
    output_up = []
    output_all_up = []
    
    for nam in col_names_up:
        output_up.append(df[nam].values[1:].tolist())
        
        for idx in list(df[nam].values[1:].tolist()):
            output_all_up.append(idx)
        
    output_lo = []
    output_all_lo = []
    
    for nam in col_names_lo:
        output_lo.append(df[nam].values[1:].tolist())
        
        for idx in list(df[nam].values[1:].tolist()):
            output_all_lo.append(idx)
        
        
    output_record[s] = output_up
    output_record[s+1] = output_lo
    output_doe[s] = output_all_up
    output_doe[s+1] = output_all_lo

In [8]:
output = pd.DataFrame(output_record)
#output.to_csv('RotaryBed.csv')

### Scale 

Scale with y = (y-minimum)/(max_y-minimum)

In [9]:
output_record_scale = collections.defaultdict(list)
output_doe_scale = {}

for i in range(10):
    for idx in range(14):
        col = np.asarray(output_record[i][idx])
        max_col, min_col = max(col), min(col)
        col = (col-min_col)/(max_col-min_col)
        output_record_scale[i].append(col)
        
    output_doe_scale[i] = []
    for idx2 in range(14):
        output_doe_scale[i] = np.concatenate([output_doe_scale[i], output_record_scale[i][idx2]])

In [10]:
output_record_stand = collections.defaultdict(list)
output_doe_stand = {}

for i in range(10):
    for idx in range(14):
        col = np.asarray(output_record[i][idx])
        avg_col = sum(col)/len(col)
        col = (col-avg_col)/avg_col
        output_record_stand[i].append(col)
        
    output_doe_stand[i] = []
    for idx2 in range(14):
        output_doe_stand[i] = np.concatenate([output_doe_stand[i], output_record_stand[i][idx2]])

### draw figures

In [ ]:
def plot_data(output, scena, measure_number, measure_label, para_label1, para_label2):
    sense = []

    plt.plot(timeSet, output[scena][measure_number], linewidth=2, color='red', label=para_label1)
    plt.plot(timeSet, output[scena+1][measure_number], linewidth=2, color='blue', label=para_label2)
    plt.xlabel('Time (min)')
    plt.ylabel(measure_label)
    plt.legend()
    plt.show()



In [ ]:
#print(df1.iloc[0])
col_names_unit = [
                  'Ads.gas_inlet.F (kmol/hr)', 
                  'Ads.gas_outlet.F (kmol/hr)', 
                  'Ads.gas_outlet.T (C)', 
                  'Ads.gas_outlet.z("CO2") (kmol/kmol)', 
                  'Des.gas_inlet.F (kmol/hr)', 
                  'Des.gas_outlet.F (kmol/hr)', 
                  'Des.gas_outlet.T (C)', 
                  'Des.gas_outlet.z("CO2") (kmol/kmol)', 
                  'Ads.T_g.Value(19,10) (C)', 
                  'Ads.T_g.Value(23,10) (C)', 
                  'Ads.T_g.Value(28,10) (C)', 
                  'Ads.z("CO2",19,10) (kmol/kmol)', 
                  'Ads.z("CO2",23,10) (kmol/kmol)', 
                  'Ads.z("CO2",28,10) (kmol/kmol)']

In [ ]:
for me, mename in enumerate(col_names_unit):

    plot_data(output_record_stand, 0, me, mename, 'MTC+3%', 'MTC-3%')
    plot_data(output_record_stand, 2, me, mename, 'HTC+3%', 'HTC-3%')
    plot_data(output_record_stand, 4, me, mename, 'DH+3%', 'DH-3%')
    plot_data(output_record_stand, 6, me, mename, 'Iso1+1%', 'Iso1-1%')
    plot_data(output_record_stand, 8, me, mename, 'Iso2+1%', 'Iso2-1%')

### DoE to get Q matrix

In [11]:
# total measurements 
num_measure = 14 
num_time = 2201
num_overall = num_measure*num_time
num_para = 5
print(num_overall)

30814


In [12]:
para_set = ['MTC', 'HTC', 'DH', 'ISO1', 'ISO2']
def FIM_Compute(output, output_region):
    # overall Q
    Q = np.zeros((num_overall,num_para))
    # Q as a set. Q[key]: [meausre1, ..., meausre14].
    Q_region = collections.defaultdict(list)

    for i in [0,2,4,6,8]:
        # first 3 parameters
        if i in [0,2,4]:
            for k in range(num_measure):
                measure1 = []
                for n in range(num_time):
                    measure1.append((output_region[i][k][n]-output_region[i+1][k][n])/0.06)
                Q_region[para_set[int(i/2)]].append(measure1)
            
            for j in range(num_overall):
                Q[j][int(i/2)] = (output[i][j]-output[i+1][j])/0.06
                
        else:
            for k in range(num_measure):
                measure1 = []
                for n in range(num_time):
                    measure1.append((output_region[i][k][n]-output_region[i+1][k][n])/0.02)
                Q_region[para_set[int(i/2)]].append(measure1)

            for l in range(num_overall):
                Q[l][int(i/2)] = (output[i][l]-output[i+1][l])/0.02
                
    FIM = Q.T@Q
    
    return Q, Q_region, FIM
        

In [13]:
Q_overall, Q_overall_region, FIM_sat = FIM_Compute(output_doe, output_record)
Q_overall_scale, Q_overall_region_scale, FIM_sat_scale = FIM_Compute(output_doe_scale, output_record_scale)
Q_overall_stand, Q_overall_region_stand, FIM_sat_stand = FIM_Compute(output_doe_stand, output_record_stand)

print(FIM_sat)

print('Determinant:', np.linalg.det(FIM_sat))
print('Eigenvalue:', np.linalg.eigvals(FIM_sat))

[[ 1.05868409e+06  5.21938490e+04  9.55467118e+05 -1.49536421e+06
  -4.81026763e+05]
 [ 5.21938490e+04  5.46070285e+03  3.21612116e+04 -4.50572524e+03
  -4.92638816e+04]
 [ 9.55467118e+05  3.21612116e+04  2.82178266e+06 -4.84129912e+06
  -3.99230566e+03]
 [-1.49536421e+06 -4.50572524e+03 -4.84129912e+06  1.03533411e+07
  -3.99686164e+05]
 [-4.81026763e+05 -4.92638816e+04 -3.99230566e+03 -3.99686164e+05
   8.38780085e+05]]
Determinant: 2.7205162168795915e+27
Eigenvalue: [1.29884340e+07 1.42166637e+06 1.35637743e+03 2.83651454e+05
 3.82940475e+05]


In [ ]:
print(len(Q_overall_region['HTC'][2]))

### Draw figures 

In [ ]:
num_time = [3,5,10,22,44,88,110,220,440,1100,2200] # per time points
para_set = ['MTC', 'HTC', 'DH', 'ISO1', 'ISO2']

In [ ]:
# extract Q 
def Qgenerator(Q_select, time_set):
    curr_Q = collections.defaultdict(list)
    
    for i in range(num_para):
        for k in range(num_measure):
            for n in time_set:
                curr_Q[para_set[i]].append(Q_select[para_set[i]][k][int(n)])
        
    return curr_Q

def FIM_calculator(Q_input, export=False):
    #print(Q_input.values())
    Q_input = np.asarray(list(Q_input.values()))
    #print(Q_input)
    #print(np.shape(Q_input))
    
    FIM = Q_input@Q_input.T
    print(np.shape(FIM))
    
    if export:
        print('unaccomplished')
        
    trace = np.trace(FIM)
    det = np.linalg.det(FIM)
    min_eig = min(np.linalg.eigvals(FIM))
    
    return trace, det, min_eig, FIM
    

In [ ]:
# get timepoints 
timeset = [[1000, 1500, 2000],
          [500, 1000, 1500, 2000, 2200]]

for i in num_time[2:]:
    timeset.append(np.linspace((2200/i),2200,i))
    
#print(timeset)

In [ ]:
def GetSet(Q_select):
    det_set = []
    trace_set = []
    mineig_set = []

    for n, num in enumerate(num_time):

        Q_iter = Qgenerator(Q_select, timeset[n])
        trace_iter, det_iter, mineig_iter, fim_iter = FIM_calculator(Q_iter)
        det_set.append(det_iter)
        trace_set.append(trace_iter)
        mineig_set.append(mineig_iter)
        
    return trace_set, det_set, mineig_set

In [ ]:
trace_set_ori, det_set_ori, mineig_ori = GetSet(Q_overall_region)

In [ ]:
trace_set_scale, det_set_scale, mineig_scale = GetSet(Q_overall_region_scale)

In [ ]:
trace_set_stand, det_set_stand, mineig_stand = GetSet(Q_overall_region_stand)

In [ ]:
print(trace_set_ori)
print(det_set_ori)
print(mineig_ori)

In [ ]:
print(np.log10(trace_set_stand))

In [ ]:
def plot_data(y_ori, y_scale, y_stand, design_cri_label):
    sense = []

    plt.scatter(np.log10(num_time), np.log10(y_ori), s=50, color='red', label='Original')
    plt.scatter(np.log10(num_time), np.log10(y_scale), marker='v', s=50, color='blue', label='Scaled')
    plt.scatter(np.log10(num_time), np.log10(y_stand), marker='*', s=50, color='green', label='Standardized')
    plt.xlabel('log10(No. of time points per measurement)')
    plt.ylabel('log10(' + design_cri_label + ')')
    plt.legend()
    plt.show()

    

In [ ]:
plot_data(mineig_ori, mineig_scale, mineig_stand, 'D-optimality')

### Data export

In [ ]:
Q_set = {}
para_set = ['MTC', 'HTC', 'DH', 'ISO1', 'ISO2']

for i in range(num_para):
    Q_set[para_set[i]] = Q_overall[:, i]

In [ ]:
Q_dict = pd.DataFrame(Q_set)
Q_dict.to_csv('Q_overall.csv')

In [ ]:
# Q_set10 accepts 220 timepoints/measurement, equally distributed. In total there should be 220*14 = 3080 
Q_set10 = collections.defaultdict(list)

time_set_overall = []
for i in range(14):
    
    time_set = np.linspace(2200*i+10+i, 2200*(i+1)+i, 220)
    for k in range(len(time_set)):
        time_set_overall.append(time_set[k]) 
print(time_set_overall)

for i in range(5):
    for k in time_set_overall:
        Q_set10[para_set[i]].append(Q_overall[int(k)][i])
    
print(len(Q_set10['MTC']))

In [ ]:
Q_dict10 = pd.DataFrame(Q_set10)
print(Q_dict10)
Q_dict10.to_csv('Q10.csv')

In [ ]:
# Q_set10 accepts 220 timepoints/measurement, equally distributed. In total there should be 220*14 = 3080 
Q_set100 = {}

time_set100 = []
for i in range(14):
    time_set = np.linspace(2200*i+100+i, 2200*(i+1)+i, 22)
    for k in range(len(time_set)):
        time_set100.append(time_set[k])
print(time_set100)

for i in range(5):
    
    Q_set100[para_set[i]] = [Q_overall[int(k),i] for k in time_set100]
    
print(len(Q_set100['MTC']))



In [ ]:
Q_dict100 = pd.DataFrame(Q_set100)
Q_dict100.to_csv('Q100.csv')

### Data export with Q_region

In [15]:
def Qgenerator(time_set, Q_region):
    curr_Q = collections.defaultdict(list)
    
    for i in range(num_para):
        for k in range(num_measure):
            for n in time_set:
                curr_Q[para_set[i]].append(Q_region[para_set[i]][k][int(n)])
        
    return curr_Q



In [21]:
#time_set = [100, 150, 200]
#time_set = [500, 1000, 1500, 2000, 2200]
time_set = np.linspace(100, 2200, 22)

Q_set5 = Qgenerator(time_set, Q_overall_region_scale)



In [22]:
Q_dict5 = pd.DataFrame(Q_set5)
print(Q_dict5)
Q_dict5.to_csv('Q22_scale.csv')

          MTC       HTC        DH      ISO1      ISO2
0   -0.539637 -0.002254 -0.049863  1.935160  0.551328
1   -0.334076  0.102165  0.101694  2.340386  0.221940
2   -0.349466  0.112066  0.255090  2.002761  0.180977
3   -0.467880  0.078989  0.301235  1.685234  0.370280
4   -0.348707  0.154449  0.367953  2.016854  0.188922
..        ...       ...       ...       ...       ...
303  1.156187  0.029326  0.442719  0.442734 -1.907701
304  0.792538  0.033848  0.295753  0.586278 -1.319648
305  0.349583  0.036776  0.142672  0.760133 -0.663229
306 -0.027378  0.035844  0.030135  0.871057 -0.131635
307 -0.287246  0.031342 -0.034661  0.886191  0.227353

[308 rows x 5 columns]


In [19]:
Q_set10 = collections.defaultdict(list)

time_set = np.linspace(100, 2200, 22)
print(time_set)

for i in range(num_para):
    for k in range(num_measure):
        for n in time_set:
            Q_set10[para_set[i]].append(Q_overall_region[para_set[i]][k][int(n)])
        
print(Q_set10)

[ 100.  200.  300.  400.  500.  600.  700.  800.  900. 1000. 1100. 1200.
 1300. 1400. 1500. 1600. 1700. 1800. 1900. 2000. 2100. 2200.]
defaultdict(<class 'list'>, {'MTC': [-2.333333333333106, -3.350000000000364, -3.7666666666666515, -3.5333333333331516, -3.2999999999996517, -3.0333333333336063, -2.7833333333328865, -1.6999999999995907, -0.5666666666665303, 0.3500000000002501, -0.04999999999976505, -0.5833333333328028, -1.066666666667023, -1.5499999999993483, -1.9333333333330909, -2.2499999999998486, -3.616666666666409, -4.166666666666667, -3.9000000000006216, -3.7333333333331593, -3.7166666666659394, -3.3833333333338564, -4.0666666666661895, -4.616666666666447, -4.63333333333272, -4.083333333333409, -3.666666666666174, -3.383333333332909, -3.1833333333338487, -2.4666666666661285, -2.116666666666826, -2.5833333333328787, -4.033333333333644, -5.21666666666647, -5.7333333333341825, -5.89999999999975, -5.7333333333341825, -5.533333333334175, -5.649999999999977, -5.09999999999972, -4.466666

In [20]:
Q_dict10 = pd.DataFrame(Q_set10)
print(Q_dict10)
Q_dict10.to_csv('Q22.csv')

          MTC       HTC         DH       ISO1      ISO2
0   -2.333333  0.000000 -12.066667  33.000000  0.950000
1   -3.350000  0.033333 -11.533333  31.100000  1.800000
2   -3.766667  0.000000 -11.066667  29.450000  2.100000
3   -3.533333  0.000000 -10.966667  29.650000  2.000000
4   -3.300000  0.200000 -10.766667  30.400000  1.600000
..        ...       ...        ...        ...       ...
303  0.009495  0.000482   0.017537  -0.023765 -0.023940
304  0.006902  0.000387   0.015330  -0.022400 -0.018055
305  0.003160  0.000315   0.013253  -0.020695 -0.011410
306 -0.000172  0.000237   0.011757  -0.019610 -0.006010
307 -0.002527  0.000150   0.010892  -0.019485 -0.002355

[308 rows x 5 columns]


### Test

In [ ]:
print(Q_overall[28713])
print(Q_overall_region['DH'][13][10])
print(Q_set['DH'][28713])
print(Q_set10['DH'][2860])
print(Q_set100['DH'][286])